In [35]:
import pandas as pd
import numpy as np

In [36]:
game_df = pd.read_csv("raw_data/Twitch_game_data.csv")
global_df = pd.read_csv("raw_data/Twitch_global_data.csv")
streamers_df = pd.read_csv("raw_data/twitchdata-update.csv")

In [37]:
def dateToDateTime(dataframe):
    df = dataframe.copy()
    df["day"] = 1
    df = df.set_index(pd.to_datetime(df[["year", "Month", "day"]]))
    df = df.drop(["year", "Month", "day"], axis=1)
    return df

## Global viewers chart

In [38]:
Viewers = global_df[["year","Month","Avg_viewers"]]

In [39]:
Viewers = dateToDateTime(Viewers)

In [40]:
Viewers.to_csv("data/global_viewers.csv")

## Games views for ridgeplot

In [41]:
game_df.drop(game_df[game_df["Game"] == "Just Chatting"].index, inplace=True)

In [42]:
top_games = game_df.groupby(['Game'])['Avg_viewers'].sum().sort_values(ascending=False)[:10].index.to_list()

In [43]:
games = []

for game in top_games:
    games.append(game_df[game_df["Game"] == game])

games = pd.concat(games)

In [44]:
games["day"] = 1
games["date"] = pd.to_datetime(games[["Year", "Month", "day"]])
games = games.drop(["Year", "Month", "day"], axis=1)
games

,Rank,Game,Hours_watched,Hours_Streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio,date
0,1,League of Legends,94377226,1362044 hours,530270,2903,129172,127021,1833,69.29,2016-01-01
200,1,League of Legends,93154772,1266715 hours,475784,2712,117996,134035,1822,73.54,2016-02-01
400,1,League of Legends,94514511,1264029 hours,599114,2585,117734,127206,1701,74.77,2016-03-01
600,1,League of Legends,88389049,1217250 hours,553165,2945,113251,122933,1692,72.61,2016-04-01
800,1,League of Legends,80679320,1196096 hours,433005,2635,118593,108585,1609,67.45,2016-05-01
...,...,...,...,...,...,...,...,...,...,...,...
13004,5,VALORANT,80581440,3217503 hours,252386,7691,277299,112074,4474,25.04,2021-06-01
13203,4,VALORANT,80880189,3289089 hours,296686,6844,287260,108856,4426,24.59,2021-07-01
13405,6,VALORANT,77525514,3085069 hours,294465,6435,272547,104341,4152,25.13,2021-08-01
13603,4,VALORANT,92393642,2972494 hours,732010,7148,257779,128502,4134,31.08,2021-09-01


In [45]:
games[3]["Game"].iloc[0]

KeyError: 3

for i,game in enumerate(games):
    game.to_csv("data/"+ games[i]["Game"].iloc[0] +".csv")